# Explore the effects of office location and year founded

In [1]:
from modules_mitch import *

ModuleNotFoundError: No module named 'modules_mitch'

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
# connect to mysql db, read cb_offices table as dataframe, disconnect
conn = dbConnect()
offices = dbTableToDataFrame(conn, 'cb_offices')
ipos = dbTableToDataFrame(conn, 'cb_ipos')
conn.close()

#### Look at offices and ipos dataframes & merge them

In [ ]:
offices.head()

In [ ]:
ipos.head()

In [ ]:
offices_ipos = pd.merge(offices, ipos, on='object_id')

In [ ]:
offices_ipos.head()

### Look at distribution of valuation amounts where provided

The valuations are all 0.2e11 or less except for 3 that are 1e11+.

In [ ]:
# plot valuation distribution; include number of companies considering
val = pd.to_numeric(offices_ipos[~offices_ipos.valuation_amount.isnull()].valuation_amount)
a=sns.distplot(val, rug=True);
a.set_title('valuation distribution (N = ' + str(len(val)) + ')');

We look at these 3 outliers and see that one is simply because the valuation amount is in JPY (currently 1 JPY = 0.0088 USD). The other two are Facebook and Amazon, which we know to be companies with very high valuationos. We will look at those with valuation_currency_code 'USD' moving forward.

In [ ]:
offices_ipos[pd.to_numeric(offices_ipos.valuation_amount) > 0.8e11]

### Look at valuation amount by region

We want to determine the effect of location on IPO valuation. In particular, consider the median valuation and number of IPOs by region.

In [ ]:
# look distribution of valuation amounts
offices_ipos_us = offices_ipos[offices_ipos.valuation_currency_code == 'USD']
val_us = pd.to_numeric(offices_ipos_us[~offices_ipos_us.valuation_amount.isnull()].valuation_amount)
region = offices_ipos_us[~offices_ipos_us.valuation_amount.isnull()].region
df_region_val = pd.concat([region, val], axis=1)
df_region_val.describe()

# look at valuation by region
#a=sns.boxplot(df_region_val.region, df_region_val.valuation_amount)
#a.set_ylabel('valuation amount')
#a.set_title('valuation by region')
#a.set_xticklabels(labels=df_region_val.region, rotation=90);

In [ ]:
# perform calculations on a per region basis
reg = df_region_val.groupby(['region'])

In [ ]:
# median ipo valuation by region
ave = reg.median()
ave.sort_values(by='valuation_amount', ascending=False)[0:4]

SF Bay has the most IPOs with NY, London, and Seattle following. 

In [ ]:
# sort regions by number of ipos (that had valutation amount) 
num = reg.count()
num.sort_values(by='valuation_amount', ascending=False)[0:10]

### Analysis of number of IPOs over time

In addition to looking at the value and region of companies that IPO we also look at historical trends in terms of the number of IPOs over time.

In [ ]:
# plot number of ipos by year with different color for each region
# now don't filter to ones with valuation amount only
dt = pd.to_datetime(offices_ipos.public_at)
valAll = pd.to_numeric(offices_ipos.valuation_amount)
regionAll = offices_ipos.region

In [ ]:
df_region_dt = pd.concat([regionAll, dt], axis=1)
time = df_region_dt.groupby([dt.dt.year])

In [ ]:
# top counts by year
num = time.count()
num.sort_values(by='public_at', ascending=False).head(10)

In [ ]:
# barplot showing number of ipos by year
a=sns.barplot(num.public_at.index, num.public_at)
a.set_ylabel('number of ipos')
a.set_xlabel('year')
a.set_title('number of ipos by year');
a.set_xticklabels(labels=num.public_at.index, rotation=90);

### Incorporate cb_ojects into analysis

While we had been considering valuation amount from the IPOs data, we ultimately decided to instead consider funding_total_usd for the below analysis because it is a much richer dataset. We have N=27,874 instead of N=167.

In [ ]:
conn = dbConnect()
objs = dbTableToDataFrame(conn, 'cb_objects')
conn.close()

In [ ]:
objs.head()

In [ ]:
objs.columns

In [ ]:
objs.shape

In [ ]:
len(objs.funding_total_usd[~objs.funding_total_usd.isnull()])

In [ ]:
# plot mean funding_total_usd by state
objs_sorted = objs[~objs.funding_total_usd.isnull()].sort_values(by='funding_total_usd', ascending=False)
a=sns.barplot(objs_sorted['state_code'], pd.to_numeric(objs_sorted['funding_total_usd']), ci=False);
a.set_xticklabels(objs_sorted['state_code'], rotation=90);

In [ ]:
# plot mean funding_total_usd by country
objs_sorted = objs[~objs.funding_total_usd.isnull()].sort_values(by='funding_total_usd', ascending=False)
a=sns.barplot(objs_sorted['country_code'], pd.to_numeric(objs_sorted['funding_total_usd']), ci=False);
a.set_xticklabels(objs_sorted['country_code'], rotation=90);

In [ ]:
# group by date founded to do analysis on the emergence of new companies over time
dt = pd.to_datetime(objs.founded_at)
df_fund_dt = pd.concat([objs.funding_total_usd, dt], axis=1)
founded = df_fund_dt.groupby([dt.dt.year])

In [ ]:
# top counts by year
num_found = founded.count()
num_found.sort_values(by='founded_at', ascending=False)['founded_at'].head(10)

In [ ]:
# plot number of new companies by year
a=sns.barplot(num_found.founded_at.index, num_found.founded_at)
a.set_ylabel('number of new companies')
a.set_xlabel('year')
a.set_title('number of new companies by year');
a.set_xticklabels(labels=num_found.founded_at.index, rotation=90);

In [ ]:
# company lifespan: closed_at - founded_at
start = pd.to_datetime(objs.founded_at)
end = pd.to_datetime(objs.closed_at)
life = end.dt.year - start.dt.year

In [ ]:
a=sns.distplot(life[~life.isnull()]);
a.set_xlim(-5, 50);

In [ ]:
# there are 27 with a life < 0 showing data entry for this data set was flawed. 
life[life < 0].describe()

In [ ]:
# consider those with positive life
life_pos = life[life>0]
life_pos[~life_pos.isnull()].describe()

In [ ]:
# see whether companies tend to have longer of shorter lifespans based on when they were founded
a=sns.barplot(start[~life.isnull()].dt.year, life[~life.isnull()], ci=None);
a.set_xticklabels(start[~life.isnull()].dt.year, rotation=90);

In [ ]:
# see whether companies tend to have longer or shorter lifespans based on region
a=sns.barplot(objs.state_code[~objs.state_code.isnull()], life[~objs.state_code.isnull()], ci=None);
a.set_xticklabels(objs.state_code[~objs.state_code.isnull()], rotation=90);